In [1]:
from obspy import UTCDateTime
from backend.common.stream_to_kafka import sendMsg
from unet_predict_pb import predict
from pwave_picking import loadStream, preprocessStream, cutStream, parseStream, saveCutStream
import backend.common.database as database

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
# 暂时读取有p波的30s波形(100HZ, 3001samples)，模拟P波拾取
stream_file = "./seed/PWU_20080720223030_30s.mseed"
# stream_file = "/Users/cui/Project/EIRS/mseed/AU.SDAN.00.BHE.202104200818.mseed"
stream = loadStream(stream_file)
stream = stream.normalize()
for tr in stream:
    print(tr.get_id()[:-3])


print(" + Preprocess stream {}".format(stream))
stream = preprocessStream(stream)
print(" -- Stream is ready, starting detection")

print(stream)

XX.PWU..
XX.PWU..
XX.PWU..
 + Preprocess stream 3 Trace(s) in Stream:
XX.PWU..BHE | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples
XX.PWU..BHN | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples
XX.PWU..BHZ | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples
 -- Stream is ready, starting detection
3 Trace(s) in Stream:
XX.PWU..BHE | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples
XX.PWU..BHN | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples
XX.PWU..BHZ | 2008-07-20T22:30:30.000000Z - 2008-07-20T22:31:00.000000Z | 100.0 Hz, 3001 samples


In [3]:
# 返回p波到时timestamp
tpstamp = predict(stream)
print("p-wave timestamp: ", UTCDateTime(tpstamp))

p-wave timestamp:  2008-07-20T22:30:54.975000Z


In [4]:
# 读取3分钟波形
stream_original = loadStream("./seed/PWU20080720223000.mseed")
print(stream_original)

3 Trace(s) in Stream:
XX.PWU..BHE | 2008-07-20T22:30:00.000000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 12001 samples
XX.PWU..BHN | 2008-07-20T22:30:00.000000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 12001 samples
XX.PWU..BHZ | 2008-07-20T22:30:00.000000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 12001 samples


In [10]:
# 截取p波到时前n秒后m秒波形
db = database.DB()
pwave_before, pwave_after = db.getPWaveConfig()
print(pwave_before, ", ", pwave_after)
stream_cut = cutStream(stream_original, tpstamp, pwave_before, pwave_after)
print(stream_cut)

40 ,  160
3 Trace(s) in Stream:
XX.PWU..BHE | 2008-07-20T22:30:14.980000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 10503 samples
XX.PWU..BHN | 2008-07-20T22:30:14.980000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 10503 samples
XX.PWU..BHZ | 2008-07-20T22:30:14.980000Z - 2008-07-20T22:32:00.000000Z | 100.0 Hz, 10503 samples


In [13]:
# 将stream解析成json
dic = parseStream(stream_cut, tpstamp)
dic.keys()

KeyError: 'XX.PWU..BHE'

In [15]:
# 向kafka发送消息
sendMsg("test", "dic")

In [10]:
# 存储剪切后的波形
saveCutStream(stream_cut, "/Users/cui/Desktop")

In [7]:
# 存储P波json至文件
import json
jsObj = json.dumps(dic)

with open ('/Users/cui/Desktop/pwave.json','w',encoding='utf8') as fp:
    fp.write(jsObj)

In [14]:
with open('/Users/cui/Desktop/pwave.json','r',encoding='utf8')as fp:
    data = dict(json.load(fp))
    print(data.keys())
    print(data['network'])


dict_keys(['network', 'station', 'pwave_timestamp', 'XX.PWU..BHE', 'XX.PWU..BHN', 'XX.PWU..BHZ'])
XX
